**Urban sprawl Purwokerto, Indonesia 1988-2011**

In [ ]:
#libraries
import pandas as pd
import os
import fiona
import rasterio as rio
import numpy as np
from PIL import Image

#custom modules
from scripts.data_acquisition import getProductUrls_Google,downloadlProducts_Google
from scripts.misc_functions import getFilePath,listUrlFromDataframe,clipRaster, normalize

#create/ specify data and output directories
datadir='./data/'
scenedir='./scenes/'
outputdir='./output/'
os.makedirs(scenedir,exist_ok=True)
os.makedirs(outputdir,exist_ok=True)

#specify Google Chrome driver executable
chrome_driver='./chromedriver_win32/chromedriver.exe'


In [ ]:
#google's landsat collection (its over 4gb)
google_scenes = pd.read_csv('https://storage.googleapis.com/gcp-public-data-landsat/index.csv.gz', compression='gzip',dtype='unicode')

#retrieve the the whole Landsat 5 archive from 1988-2011 for Purwokerto and return the product urls (Google Cloud) 
product_url = getProductUrls_Google(google_scenes,datadir,path=120,row=65)

In [ ]:
#or load in the csv file containg the product url
csv_file = getFilePath(datadir,None,'l5_df.csv')
product_df = pd.read_csv(csv_file,index_col=0)
product_url = listUrlFromDataframe(product_df)

In [ ]:
#download data
#specify authentication to access Google Cloud via script
import getpass
email='william.live94@gmail.com'
password=getpass.getpass()

lt_data=['B3','B2','B1']
for url in product_url:
    downloadlProducts_Google(url,scenedir,chrome_driver,lt_data,email,password)

In [ ]:
#clip data based on boundary
pwt_bound=getFilePath(datadir,'pwt_shp','.shp')
with fiona.open(pwt_bound, 'r') as shapefile:
    features = [feature["geometry"] for feature in shapefile]
    
for product in os.listdir(scenedir):
    product_dir = os.path.join(scenedir,product)
    for file in os.listdir(product_dir):
        image_file = os.path.join(product_dir,file)
        band_nr = file.split('_')[-1].split('.')[0]
        file_name = '{}_clip.TIF'.format(band_nr)
        clipRaster(image_file,features,product_dir,file_name)

In [ ]:
import glob
for product in os.listdir(scenedir):
    product_dir = os.path.join(scenedir,product)
    red= glob.glob(product_dir+'/B3_clip.TIF')[0]
    green = glob.glob(product_dir+'/B2_clip.TIF')[0]
    blue = glob.glob(product_dir+'/B1_clip.TIF')[0]
    
    with rio.open(red,'r') as src_red:
        image_red = src_red.read(1)
    with rio.open(green,'r') as src_green:
        image_green = src_green.read(1)
    with rio.open(blue,'r') as src_blue:
        image_blue = src_blue.read(1)
    
    rgb=np.dstack((image_red,image_green,image_blue))
    rgb =normalize(rgb,255)
    rgb = rgb.astype(rio.uint8)
    
    file_name=os.path.basename(product_dir+"_rgb.png")
    file = os.path.join(outputdir,file_name)
    img = Image.fromarray(rgb,'RGB')
    img.save(file)

